# Installing Scikit Learn

In [4]:
!pip3 install -U scikit-learn scipy matplotlib

Requirement already up-to-date: scipy in c:\users\nilsvn\appdata\local\programs\python\python36-32\lib\site-packages (1.4.1)
Requirement already up-to-date: matplotlib in c:\users\nilsvn\appdata\local\programs\python\python36-32\lib\site-packages (3.2.1)
Could not build wheels for scipy, since package 'wheel' is not installed.
Could not build wheels for matplotlib, since package 'wheel' is not installed.
Could not build wheels for numpy, since package 'wheel' is not installed.
Could not build wheels for cycler, since package 'wheel' is not installed.
Could not build wheels for kiwisolver, since package 'wheel' is not installed.
Could not build wheels for python-dateutil, since package 'wheel' is not installed.
Could not build wheels for pyparsing, since package 'wheel' is not installed.
Could not build wheels for six, since package 'wheel' is not installed.


You should consider upgrading via the 'c:\users\nilsvn\appdata\local\programs\python\python36-32\python.exe -m pip install --upgrade pip' command.


In [338]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time
from sklearn.preprocessing import StandardScaler


In [343]:
def get_df(normal_path, attack_path_1, attack_path_2):
    df_normal = pd.read_csv(normal_path)
    df_attack_1 = pd.read_csv(attack_path_1)
    df_attack_2 = pd.read_csv(attack_path_2)
    
    df_normal['Class'] = 0 #create new 'Class' column containing only 0's.
    df_attack_1['Class'] = 1 #create new 'Class' column containing only 1's.
    df_attack_2['Class'] = 1 #create new 'Class' column containing only 1's.
    
    # feel free to change the amount depending on the amount of RAM you have (max 16,906).
    df_normal_1 = df_normal.head(5000)
    df_normal_2 = df_normal.tail(5000)
    
    df_train = pd.concat([df_attack_1, df_normal_1], ignore_index=True)
    df_test = pd.concat([df_attack_2, df_normal_2], ignore_index=True)
    return df_train, df_test

In [344]:
def get_X_y(df):
    X = df.drop('Class', axis=1)
    y = df['Class']
    return X, y

In [369]:
def SVM(X_train, X_test, y_train, y_test, threshold):
    model= svm.SVC(kernel='linear', probability=True)
    model.fit(X_train, y_train)
    y_pred = (model.predict_proba(X_test)[:,1] >= threshold).astype(bool)
    
    print(metrics.confusion_matrix(y_test, y_pred))
    print(metrics.classification_report(y_test, y_pred, digits=3))
    
    ACC = metrics.accuracy_score(y_test, y_pred)
    PRC = metrics.precision_score(y_test, y_pred) # when multiclass: average=None
    REC = metrics.recall_score(y_test, y_pred) # when multiclass: average=None
    return ACC, PRC, REC
    

In [370]:
def datetime_string_to_epoch(str):
    format = '%Y-%m-%d %H:%M:%S'
    if str[10] == 'T':
        format = '%Y-%m-%dT%H:%M:%S'
    t = time.strptime(str[:19], format)
    return time.mktime(t)

In [371]:
def preprocess_data(df):
    # filter to keep most relevant columns
    df = df[['TimeGenerated', 'ConditionalAccessStatus', 'Class']] # 'CorrelationId',
    # convert datetime strings to epoch
    df['TimeGenerated'] = df['TimeGenerated'].map(datetime_string_to_epoch)
    # split Pandas DataFrame into X (data) and y (corresponding class labels)
    X, y = get_X_y(df)
    # One-Hot Encoding for the nominal categorical variables
    X = pd.get_dummies(X[['TimeGenerated', 'ConditionalAccessStatus']]) # 'CorrelationId',
    # Standardization of values
    std = StandardScaler()
    X['TimeGenerated'] = std.fit_transform(X[['TimeGenerated']])
    
    return X, y

In [406]:
# change these file paths when reproducing results
# files are available on my github: https://github.com/NilsvN/my_thesis
NORMAL_PATH = "C:/Users/NilsvN/Desktop/Research/Azure/30_day_login/30days_anonymized_sigin_logs_column_corrected.csv"
ATTACK_PATH_1 = "C:/Users/NilsvN/Desktop/Research/Azure/Attacks/01_brute_force/brute_force_1.csv"
ATTACK_PATH_2 = "C:/Users/NilsvN/Desktop/Research/Azure/Attacks/01_brute_force/brute_force_2.csv"

# create df's from csv data
df_train, df_test = get_df(NORMAL_PATH, ATTACK_PATH_1, ATTACK_PATH_1)
X_train, y_train = preprocess_data(df_train)
X_test, y_test = preprocess_data(df_test)
ACC, PRC, REC = SVM(X_train, X_test, y_train, y_test, 0.00078)
# for SVM optimal threshold lies between 0.0010 and 0.0001
# attacks detected as attacks: turning point between 0.00078 and 0.00079
print('ACC: ' + str(ACC))
print('PRC: ' + str(PRC))
print('REC: ' + str(REC))

c:\users\nilsvn\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\nilsvn\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


[[4886  114]
 [   0   20]]
              precision    recall  f1-score   support

           0      1.000     0.977     0.988      5000
           1      0.149     1.000     0.260        20

    accuracy                          0.977      5020
   macro avg      0.575     0.989     0.624      5020
weighted avg      0.997     0.977     0.986      5020

ACC: 0.9772908366533865
PRC: 0.14925373134328357
REC: 1.0
